In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    """
    Create adversarial example using FGSM.

    Parameters:
    - image: Original input image (tensor).
    - epsilon: Perturbation amount.
    - data_grad: Gradient of the loss with respect to the input image.

    Returns:
    - perturbed_image: Adversarial image.
    """
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()

    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon * sign_data_grad

    # Adding clipping to maintain [0, 1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image

In [ ]:
def generate_adversarial_example(model, image, target_label, epsilon):
    """
    Generate an adversarial example for a given model and input image.

    Parameters:
    - model: The neural network model.
    - image: Original input image (tensor).
    - target_label: True label of the input image.
    - epsilon: Perturbation amount.

    Returns:
    - perturbed_image: Adversarial image.
    """
    # Set requires_grad attribute of tensor. Important for Attack
    image.requires_grad = True

    # Forward pass the data through the model
    output = model(image)

    # Calculate the loss
    loss = F.nll_loss(output, target_label)

    # Zero all existing gradients
    model.zero_grad()

    # Calculate gradients of model in backward pass
    loss.backward()

    # Collect data gradient
    data_grad = image.grad.data

    # Create adversarial example
    perturbed_image = fgsm_attack(image, epsilon, data_grad)

    return perturbed_image

In [ ]:
def save_image(tensor_image, filename):
    """
    Save a tensor as an image file.

    Parameters:
    - tensor_image: The tensor representation of the image (C x H x W).
    - filename: The path where the image will be saved.
    """
    # Convert tensor to PIL Image
    pil_image = transforms.ToPILImage()(tensor_image.squeeze(0))  # Remove batch dimension
    pil_image.save(filename)

In [ ]:
if __name__ == "__main__":
    # Load a pre-trained ResNet18 model
    model = models.vgg16(pretrained=True)

    # Set the model to evaluation mode
    model.eval()

    # Load and preprocess your image
    img_path = '/content/saqib.jpeg'  # Replace with your actual image path
    original_image = Image.open(img_path)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match model input size
        transforms.ToTensor(),           # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per model requirements
    ])

    image_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension

    # Define target label and epsilon value for perturbation
    target_label = torch.tensor([1])  # Replace with actual label index (e.g., class index)

    epsilon = 0.01  # Adjust this value for desired perturbation level

    # Generate adversarial example
    adv_image = generate_adversarial_example(model, image_tensor, target_label, epsilon)

    # Save the adversarial image
    save_image(adv_image, 'adversarial_example_khizar.png')  # Save as PNG file

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 88.1MB/s]


# **Code with Guassian Blur added to smoothen the image out**

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image
import numpy as np

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    """
    Create adversarial example using FGSM with a lower epsilon for subtle changes.

    Parameters:
    - image: Original input image (tensor).
    - epsilon: Perturbation amount.
    - data_grad: Gradient of the loss with respect to the input image.

    Returns:
    - perturbed_image: Adversarial image.
    """
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()

    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon * sign_data_grad

    # Adding clipping to maintain [0, 1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image

In [ ]:
def generate_adversarial_example(model, image, target_label, epsilon):
    """
    Generate an adversarial example for a given model and input image.

    Parameters:
    - model: The neural network model.
    - image: Original input image (tensor).
    - target_label: True label of the input image.
    - epsilon: Perturbation amount.

    Returns:
    - perturbed_image: Adversarial image.
    """
    # Set requires_grad attribute of tensor. Important for Attack
    image.requires_grad = True

    # Forward pass the data through the model
    output = model(image)

    # Calculate the loss
    loss = F.nll_loss(output, target_label)

    # Zero all existing gradients
    model.zero_grad()

    # Calculate gradients of model in backward pass
    loss.backward()

    # Collect data gradient
    data_grad = image.grad.data

    # Create adversarial example with reduced epsilon
    perturbed_image = fgsm_attack(image, epsilon, data_grad)

    return perturbed_image

In [ ]:
def save_image(tensor_image, filename):
    """
    Save a tensor as an image file.

    Parameters:
    - tensor_image: The tensor representation of the image (C x H x W).
    - filename: The path where the image will be saved.
    """
    # Convert tensor to PIL Image
    pil_image = transforms.ToPILImage()(tensor_image.squeeze(0))  # Remove batch dimension
    pil_image.save(filename)

In [ ]:
if __name__ == "__main__":
    # Load a pre-trained ResNet18 model
    model = models.vgg16(pretrained=True)

    # Set the model to evaluation mode
    model.eval()

    # Load and preprocess your image
    img_path = '/content/saqib.jpeg'  # Path to original image
    original_image = Image.open(img_path)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match model input size
        transforms.ToTensor(),           # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per model requirements
    ])

    image_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension

    # Define target label and reduced epsilon value for subtle perturbation
    target_label = torch.tensor([1])  # Replace with actual label index (e.g., class index)
    epsilon = 0.005  # Reduced epsilon for less visible perturbations

    # Generate adversarial example
    adv_image = generate_adversarial_example(model, image_tensor, target_label, epsilon)

    # Save the adversarial image
    save_image(adv_image, '/content/adversarial_example_saqib_blur.png')  # Save as PNG file

# **Code with PGD adversarial attack**

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image

In [ ]:
def pgd_attack(image, model, target_label, epsilon, alpha, num_steps):
    """
    Generate an adversarial example using the PGD method.

    Parameters:
    - image: Original input image (tensor).
    - model: The neural network model.
    - target_label: True label of the input image.
    - epsilon: Maximum perturbation amount.
    - alpha: Step size for each iteration.
    - num_steps: Number of steps in the PGD attack.

    Returns:
    - perturbed_image: Adversarial image.
    """
    # Clone the input image to avoid modifying the original
    perturbed_image = image.clone().detach()
    perturbed_image.requires_grad = True

    for _ in range(num_steps):
        # Forward pass the data through the model
        output = model(perturbed_image)

        # Calculate the loss
        loss = F.nll_loss(output, target_label)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Get the sign of the gradients
        data_grad = perturbed_image.grad.data

        # Update the image by taking a step in the direction of the gradient
        perturbed_image = perturbed_image + alpha * data_grad.sign()

        # Clamp the perturbation to ensure it stays within the epsilon ball
        perturbation = torch.clamp(perturbed_image - image, -epsilon, epsilon)
        perturbed_image = torch.clamp(image + perturbation, 0, 1).detach()  # Ensure values are in [0,1]
        perturbed_image.requires_grad = True  # Re-enable gradient computation for the next step

    return perturbed_image

In [ ]:
def save_image(tensor_image, filename):
    """
    Save a tensor as an image file.

    Parameters:
    - tensor_image: The tensor representation of the image (C x H x W).
    - filename: The path where the image will be saved.
    """
    # Convert tensor to PIL Image
    pil_image = transforms.ToPILImage()(tensor_image.squeeze(0))  # Remove batch dimension
    pil_image.save(filename)

In [ ]:
if __name__ == "__main__":
    # Load a pre-trained ResNet18 model
    model = models.resnet18(pretrained=True)

    # Set the model to evaluation mode
    model.eval()

    # Load and preprocess the image
    img_path = '/content/saqib.jpeg'  # Replace with your actual image path
    original_image = Image.open(img_path)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match model input size
        transforms.ToTensor(),           # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per model requirements
    ])

    image_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension

    # Define target label, epsilon, alpha, and num_steps for the PGD attack
    target_label = torch.tensor([1])  # Replace with the desired label index (e.g., class index)
    epsilon = 0.005                     # Maximum perturbation
    alpha = 0.001                      # Step size
    num_steps = 40                     # Number of steps in PGD

    # Generate adversarial example using PGD
    adv_image = pgd_attack(image_tensor, model, target_label, epsilon, alpha, num_steps)

    # Save the adversarial image
    save_image(adv_image, '/content/pgd_adversarial_example_saqib_pgd.png')

# **PGD Attack along with keeping image size same as original**

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image

In [ ]:
def pgd_attack(model, image, target_label, epsilon, alpha, num_iter):
    """
    Perform PGD attack to create adversarial example.

    Parameters:
    - model: The neural network model.
    - image: Original input image (tensor).
    - target_label: True label of the input image.
    - epsilon: Maximum perturbation.
    - alpha: Step size for each iteration.
    - num_iter: Number of iterations for PGD.

    Returns:
    - perturbed_image: Adversarial image generated using PGD.
    """
    perturbed_image = image.clone()
    perturbed_image.requires_grad = True

    for i in range(num_iter):
        # Forward pass the data through the model
        output = model(perturbed_image)

        # Calculate the loss
        loss = F.nll_loss(output, target_label)

        # Zero all existing gradients
        model.zero_grad()

        # Backward pass to calculate gradients
        loss.backward()

        # Update image with gradient step
        perturbed_image = perturbed_image + alpha * perturbed_image.grad.sign()

        # Clip perturbation to be within epsilon limit and [0,1] range
        perturbation = torch.clamp(perturbed_image - image, min=-epsilon, max=epsilon)
        perturbed_image = torch.clamp(image + perturbation, min=0, max=1).detach_()

        # Re-enable gradient computation for the next iteration
        perturbed_image.requires_grad = True

    return perturbed_image


In [ ]:
def save_image(tensor_image, filename, mean, std, original_size):
    """
    Denormalizes a tensor image, resizes it to the original size, and saves it as a PNG file.

    Parameters:
    - tensor_image: The tensor representation of the image (C x H x W).
    - filename: The path where the image will be saved.
    - mean: List of mean values used for normalization, e.g., [0.485, 0.456, 0.406].
    - std: List of standard deviation values used for normalization, e.g., [0.229, 0.224, 0.225].
    - original_size: Tuple of the original image size (width, height).
    """
    # Denormalize
    denormalized_image = tensor_image.clone()
    for c in range(denormalized_image.shape[1]):
        denormalized_image[0, c] = denormalized_image[0, c] * std[c] + mean[c]

    # Clamp to [0,1]
    denormalized_image = torch.clamp(denormalized_image, 0, 1)

    # Convert to PIL Image
    pil_image = transforms.ToPILImage()(denormalized_image.squeeze(0))  # Remove batch dimension

    # Resize to original size
    pil_image = pil_image.resize(original_size, Image.Resampling.LANCZOS)

    # Save image
    pil_image.save(filename)

In [ ]:
# Main usage example
if __name__ == "__main__":
    # Load a pre-trained ResNet18 model
    model = models.vgg16(pretrained=True)
    model.eval()  # Set the model to evaluation mode

    # Load and preprocess the input image
    img_path = '/content/saqib.jpeg'  # Replace with your actual image path
    original_image = Image.open(img_path)
    original_size = original_image.size  # Save original size (width, height)

    # Transform for the image to match model input
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to model's input size
        transforms.ToTensor(),          # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])
    image_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension

    # Parameters for PGD attack
    epsilon = 0.01   # Maximum perturbation
    alpha = 0.01    # Step size
    num_iter = 50   # Number of iterations
    target_label = torch.tensor([1])  # Replace with actual target label index

    # Generate adversarial example using PGD
    adv_image = pgd_attack(model, image_tensor, target_label, epsilon, alpha, num_iter)

    # Save the adversarial image with denormalization and resizing to original size
    save_image(adv_image, '/content/sameSize_adversarial_example_saqib1.jpeg',
               mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], original_size=original_size)